In [1]:
from pygromos.files.topology import top
from pygromos.files.topology import ptp
from pygromos.files.coord import cnf

import copy
import numpy as np

from rdkit import Chem


In [2]:
in_top1_path = "F313.top"
in_top2_path = "G078.top"
out_ptp_path = "test.ptp"

in_coord2_path = "G078_unitedatom_optimised_geometry.g96"
in_coord1_path = "F313_unitedatom_optimised_geometry.g96"

In [3]:
def generate_dual_topology_approach( eds:bool=False):
    pass

In [4]:
def generate_hybrid_topology_approach():
    pass

In [5]:
atom_ids = {}

def generate_single_topology_approach(topology:top.Top, atom_mapping:dict):


    pass

## COORD

In [7]:
cmol1 = cnf.Cnf(in_coord1_path)
cmol2 = cnf.Cnf(in_coord2_path)

In [44]:
cmol2.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [9]:
 cmol1.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Atom Matching

In [40]:
from rdkit.Chem import rdFMCS

def find_atom_mapping(cnfA:cnf.Cnf, cnfB:cnf.Cnf)->(dict,str):
    
    mol1 = Chem.MolFromPDBBlock(cmol1.get_pdb(rdkit_ready=True), removeHs=False)
    mol2 = Chem.MolFromPDBBlock(cmol2.get_pdb(rdkit_ready=True), removeHs=False)


    #CompareElements
    mcs = rdFMCS.FindMCS([mol1, mol2],
                         atomCompare=rdFMCS.AtomCompare.CompareAnyHeavyAtom,
                         bondCompare=rdFMCS.BondCompare.CompareOrderExact,
                         ringMatchesRingOnly=True, completeRingsOnly=True)
    #                    ringCompare=rdFMCS.RingCompare.StrictRingFusion)
    smartsString = mcs.smartsString

    ##MCS Match
    patt = Chem.MolFromSmarts(smartsString)  # smartsString
    mol1Match = mol1.GetSubstructMatch(patt)
    mol2Match = mol2.GetSubstructMatch(patt)

    atom_mappingAB = dict(zip(mol1Match, mol2Match))
    return atom_mappingAB, smartsString

atom_mappingAB, smart = find_atom_mapping(cnfA=mol1, cnfB=mol2)
len(atom_mappingAB), atom_mappingAB, smart

(10,
 {2: 1, 1: 0, 3: 9, 5: 8, 7: 6, 11: 4, 13: 2, 14: 3, 12: 5, 8: 7},
 '[#6&R]1(-&!@[#8,#6;!R])-&@[#6&R]-&@[#6&R]-&@[#6&R](-&@[#6&R](-&@[#6&R]-&@1-&!@[#1&!R])-&!@[#1&!R])-&!@[#7,#6;!R]')

## COORDINATE Generation
### Coord Alignment

In [32]:
def align_cnfs_with_MCS(cnfA:cnf.Cnf, cnfB:cnf.Cnf, atom_mappingAB:int)->(cnf.Cnf, cnf.Cnf):
    
    cnfA= copy.deepcopy(cnfA)
    cnfB= copy.deepcopy(cnfB)
    
    mol1 = Chem.MolFromPDBBlock(cnfA.get_pdb(rdkit_ready=True), removeHs=False)
    mol2 = Chem.MolFromPDBBlock(cnfB.get_pdb(rdkit_ready=True), removeHs=False)
    
    AllChem.AlignMol(mol1, mol2, atomMap=list(atom_mappingAB.items()))


    conf2 = mol2.GetConformer()
    for ind, pos in enumerate(conf2.GetPositions()):
        cnfB.POSITION[ind].xp= pos[0]/10
        cnfB.POSITION[ind].yp= pos[1]/10
        cnfB.POSITION[ind].zp= pos[2]/10

    conf1 = mol1.GetConformer()
    for ind, pos in enumerate(conf1.GetPositions()):
        cnfA.POSITION[ind].xp= pos[0]/10
        cnfA.POSITION[ind].yp= pos[1]/10
        cnfA.POSITION[ind].zp= pos[2]/10
        
    return cnfA, cnfB

cmol1, cmol2 = align_cnfs_with_MCS(cnfA=cmol1, cnfB=cmol2, atom_mappingAB=atom_mappingAB)



### State merging

In [34]:
eucldean_dist = lambda x,y: np.sqrt((x.xp-y.xp)**2+(x.yp-y.yp)**2+(x.zp-y.zp)**2)

def merge_states(cnfA:cnf.Cnf, cnfB:cnf.Cnf, atomMatchingAB:dict, dist_tresh:float=0.09)-> cnf.Cnf:#nm
    
    
    natoms_in_a=len(cnfA.POSITION)
    match_molA, match_molB = np.array(list(atom_mappingAB.items())).T

    cmol_comb = copy.deepcopy(cnfA)    
    
    for pos in cnfB.POSITION:
        #print(pos)
        #print(pos.atomID in selected_b_match)
        #print(any([eucldean_dist(pos, pos2) < dist_tresh for pos2 in cmol_comb.POSITION]), [eucldean_dist(pos, pos2) <dist_tresh for pos2 in cmol_comb.POSITION])
        #print([eucldean_dist(pos, pos2)  for pos2 in selected_b_cnf.POSITION])

        if(pos.atomID in match_molB or 
           any([eucldean_dist(pos, pos2) < dist_tresh for pos2 in cnfA.POSITION])):#check distance
            #print("OW!")
            continue
        else:
            #print('new')
            pos = copy.deepcopy(pos)
            pos.resID=2
            pos.atomID += natoms_in_a
            cmol_comb.POSITION.append(pos)

    #clean cmol_comb
    present_atoms = []
    for pos in cmol_comb.POSITION:
        pos.resID=1
        pos.atomType = "C"
        pos.resName=cmol1.POSITION[0].resName[:2]+cmol2.POSITION[0].resName[2:4]
        present_atoms.append(pos.atomID)
    return cmol_comb

cmol_comb = merge_states(cnfA=cmol1, cnfB=cmol2, atomMatchingAB=atom_mappingAB, dist_tresh=0.09)
cmol_comb.write("fung.g96")
cmol_comb.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [35]:
cmol_comb.POSITION

POSITION
# 	 
    1 F378  C          1   -0.213738224    0.354296869   -0.003149189
    1 F378  C          2   -0.093034159    0.279002559    0.000232480
    1 F378  C          3   -0.102927162    0.141901993    0.000715450
    1 F378  C          4    0.018276367    0.071408239   -0.000002296
    1 F378  C          5    0.111273454    0.127513078   -0.000901660
    1 F378  C          6    0.019383309   -0.067691484   -0.001720367
    1 F378  C          7    0.114585920   -0.120386532   -0.002633961
    1 F378  C          8   -0.100712998   -0.140997530   -0.002121299
    1 F378  C          9   -0.099305450   -0.281798182    0.003460389
    1 F378  C         10   -0.182306122   -0.324497236   -0.036439226
    1 F378  C         11   -0.016806384   -0.322988495   -0.039050813
    1 F378  C         12   -0.221316551   -0.069603785   -0.001103479
    1 F378  C         13   -0.315613778   -0.123808612   -0.000803777
    1 F378  C         14   -0.223123519    0.070295877    0.000814745
    1 

## Topology

In [41]:
top1 = top.Top(in_top1_path)
top2 = top.Top(in_top2_path)


In [42]:
com_top = top2
com_top += top1
red_com_top = copy.deepcopy(com_top)

In [56]:
#BONDS
bonds =[]
for bond in com_top.BOND:
    if(bond.IB in present_atoms and bond.JB in present_atoms):
        bonds.append(bond)
red_com_top.BOND.content = bonds
red_com_top.BOND.NBON = len(bonds)

#BondAngles
angles =[]
for angle in com_top.BONDANGLE:
    if(angle.IT in present_atoms and angle.JT in present_atoms and angle.KT in present_atoms):
        angles.append(angle)

red_com_top.BONDANGLE.content = angles
red_com_top.BONDANGLE.NTHE = len(angles)

#BondAnglesH
anglesH =[]
for angleH in com_top.BONDANGLEH:
    if(angleH.IT in present_atoms and angleH.JT in present_atoms and angleH.KT in present_atoms):
        anglesH.append(angle)

red_com_top.BONDANGLEH.content = anglesH
red_com_top.BONDANGLEH.NTHEH = len(anglesH)

#Dihedrals
dihedrals =[]
for dih in com_top.DIHEDRAL:
    if(dih.IP in present_atoms and dih.JP in present_atoms and dih.KP in present_atoms and dih.LP in present_atoms):
        dihedrals.append(dih)

red_com_top.DIHEDRAL.content = dihedrals
red_com_top.DIHEDRAL.NPHI = len(dihedrals)

#DihedralsH
dihedralsH =[]
for dihh in com_top.DIHEDRALH:
    if(dihh.IPH in present_atoms and dihh.JPH in present_atoms and dihh.KPH in present_atoms and dihh.LPH in present_atoms):
        dihedralsH.append(dihh)

red_com_top.DIHEDRALH.content = dihedralsH
red_com_top.DIHEDRALH.NPHIH = len(dihedralsH)

#Impropers
impdihedrals =[]
for imp in com_top.IMPDIHEDRAL:
    if(imp.IQ in present_atoms and imp.JQ in present_atoms and imp.KQ in present_atoms and imp.LQ in present_atoms):
        impdihedrals.append(dihh)

red_com_top.IMPDIHEDRAL.content = impdihedrals
red_com_top.IMPDIHEDRAL.NQHI = len(impdihedrals)

#ImpropersH
impdihedralsH =[]
for impH in com_top.IMPDIHEDRALH:
    if(impH.IQH in present_atoms and impH.JQH in present_atoms and impH.KQH in present_atoms and impH.LQH in present_atoms):
        impdihedralsH.append(dihh)

red_com_top.IMPDIHEDRALH.content = impdihedralsH
red_com_top.IMPDIHEDRALH.NQHIH = len(impdihedralsH)

#Mass, VDW, Charge
sol =[]
for atm in com_top.SOLUTEATOM:
    if(atm.ATNM in present_atoms):
        sol.append(atm)

red_com_top.SOLUTEATOM.content = sol 
red_com_top.SOLUTEATOM.NRP = len(sol)

"""
#crossDIHEDRAL
crds =[]
for crd in com_top.CROSSDIHEDRAL:
    print(vars(crd))
    if(crd.ATNM in present_atoms):
        crds.append(atm)

red_com_top.CROSSDIHEDRAL.content = crds 

#crossDIHEDRALH
crdsH =[]
for crdH in com_top.CROSSDIHEDRALH:
    print(vars(crdH))
    if(crdH.ATNM in present_atoms):
        crdsH.append(atm)

red_com_top.CROSSDIHEDRALH.content = crdsH
"""

##resname meshing
molA_Name = com_top.RESNAME.content[1][0]
molB_Name = com_top.RESNAME.content[2][0]

red_com_top.RESNAME.content = [['1'], [molA_Name[:2]+molB_Name[:2]]]


pass

### Build PTP

In [21]:
ptp_file = ptp.Ptp()

In [22]:
from pygromos.files.blocks.topology_blocks import PERTPROPERDIH, atom_lam_pertubation_state_dihedral

In [23]:
pertproperdih =PERTPROPERDIH()

In [24]:
for 
pertproperdih.content.append()

SyntaxError: invalid syntax (Temp/ipykernel_7584/1245646628.py, line 1)

In [ ]:
## DONE